In [ ]:
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.linear_model import LogisticRegression, LinearRegression
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble.partial_dependence import plot_partial_dependence

In [1]:
import pandas as pd

In [2]:
df0 = pd.read_csv('../data/churn_train.csv')

In [3]:
df = df0.copy()

In [4]:
df.head(5).T

,0,1,2,3,4
avg_dist,6.94,8.06,21.5,9.46,13.77
avg_rating_by_driver,5,5,4,5,5
avg_rating_of_driver,5,5,NaN,NaN,NaN
avg_surge,1,1,1,2.75,1
city,Astapor,Astapor,Winterfell,Winterfell,Winterfell
last_trip_date,2014-05-03,2014-01-26,2014-05-21,2014-01-10,2014-05-13
phone,Android,Android,iPhone,Android,iPhone
signup_date,2014-01-12,2014-01-25,2014-01-02,2014-01-09,2014-01-31
surge_pct,0,0,0,100,0
trips_in_first_30_days,0,2,1,1,0


In [5]:
df.head(5)

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,6.94,5.0,5.0,1.00,Astapor,2014-05-03,Android,2014-01-12,0.0,0,False,100.0
1,8.06,5.0,5.0,1.00,Astapor,2014-01-26,Android,2014-01-25,0.0,2,True,0.0
2,21.50,4.0,NaN,1.00,Winterfell,2014-05-21,iPhone,2014-01-02,0.0,1,True,100.0
3,9.46,5.0,NaN,2.75,Winterfell,2014-01-10,Android,2014-01-09,100.0,1,False,100.0
4,13.77,5.0,NaN,1.00,Winterfell,2014-05-13,iPhone,2014-01-31,0.0,0,False,100.0


In [6]:
#new_df = df['last_trip_date'].sort_values(ascending = False)
new = df.sort_values(by = ['last_trip_date'])

In [13]:
new.head(5)

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
38107,13.75,5.0,5.0,1.00,Winterfell,2014-01-01,iPhone,2014-01-01,0.0,2,False,100.0
35244,3.59,5.0,NaN,1.00,Winterfell,2014-01-01,iPhone,2014-01-01,0.0,1,True,100.0
1449,2.45,5.0,5.0,1.25,Astapor,2014-01-01,iPhone,2014-01-01,100.0,1,False,100.0
9287,4.45,5.0,2.0,1.00,Winterfell,2014-01-01,iPhone,2014-01-01,0.0,1,False,100.0
7168,18.36,5.0,4.0,1.00,Winterfell,2014-01-01,Android,2014-01-01,0.0,1,False,100.0


In [12]:
print(new['last_trip_date'][0])

2014-05-03


In [ ]:
# today = '2014-07-01'
# new['y_label'] = new['last_trip_date'] - today

In [ ]:
#new.info()
new['avg_rating_of_driver'].fillna(new['avg_rating_of_driver'].mean(), inplace = True)
new['avg_rating_by_driver'].fillna(new['avg_rating_by_driver'].mean(), inplace = True)

In [ ]:
new.info()

In [ ]:
new.dropna(axis = 0, inplace = True)

In [ ]:
new.info()

In [ ]:
df = new.copy()

In [ ]:
df['last_trip_date'] = pd.to_datetime(df['last_trip_date'])

In [ ]:
df['churn'] = df['last_trip_date'] < '2014-06-01'

In [ ]:
df['churn'] = df['churn'] * 1
df['phone'] = df['phone'] == 'iPhone'
df['phone'] = df['phone'] * 1


In [ ]:
df = pd.get_dummies(df, columns = ['city'], drop_first = False)

In [ ]:
df['luxury_car_user'] = df['luxury_car_user'] * 1

In [ ]:
df['phone'].unique()

In [ ]:
df.head(5).T

In [ ]:

X = df.drop(columns = ['signup_date','last_trip_date','churn'])


In [ ]:
y = df['churn']

In [ ]:
log_model = LogisticRegression()
log_model.fit(X_train,y_train)

In [ ]:
#X.head(5).T
y_pred = log_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y, y_pred, labels = [1,0])

In [ ]:
tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

In [ ]:
(tn, fp, fn, tp)

In [ ]:
con_matrix = np.array([[tp, fn], [fp, tn]])

In [ ]:
con_matrix

In [ ]:
log_model.coef_

In [ ]:
coeff = log_model.coef_
ind = (coeff**2).argsort()
labels = [item for item in X.columns]

In [ ]:
fig,ax = plt.subplots()
plt.axhline(y=0,color='red')
plt.plot(np.linspace(0, 11, 12), coeff.T)
ax.set_xticklabels(labels)
plt.xticks(np.linspace(0,11,12), rotation=90);

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot()
x = np.linspace(0, 11, 12)
markerline, stemlines, baseline = plt.stem(x, coeff.T, '-.')
plt.setp(baseline, color='r', linewidth=2)
plt.xticks(np.linspace(0,11,12), labels,rotation=90)
plt.xlabel('Features')
plt.ylabel('Beta Coefficients')
plt.title('Feature Impact')
plt.grid(linestyle ='--',lw=0.5)
plt.show()

In [ ]:
print(classification_report(y, y_pred))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

In [ ]:
rf_model.feature_importances_

In [ ]:
y_pred = rf_model.predict(X_test)

In [ ]:
def metrics(y, y_pred):
    print("Accuracy Score: ", accuracy_score(y, y_pred))
    print("Confusion Matrix: ", confusion_matrix(y, y_pred, labels = [1,0]))
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    print("Precision Score: ", precision_score(y, y_pred))
    print("Recall Score: ", recall_score(y, y_pred))
    print("")
    return

In [ ]:
metrics(y_test, y_pred)

In [ ]:
def model_pred(X_train, y_train, X_test, y_test, model, trees = 100):
    if model == LogisticRegression:
        model = model()
    else:
        model = model(n_estimators = trees)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_probs = model.predict_proba(X_test)[:,1]
    return y_pred, y_pred_probs

In [ ]:
# ROC Curve
def ROC_curve_plot(y_test,y_pred_probs, model):
    fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label=model)
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()
    return 
    

In [ ]:
y_pred, y_pred_probs = model_pred(X_train, y_train, X_test, y_test, RandomForestClassifier)

In [ ]:
y_pred_probs.shape

In [ ]:
ROC_curve_plot(y_test,y_pred_probs, 'RF')

In [ ]:
y_pred, y_pred_probs = model_pred(X_train, y_train, X_test, y_test, LogisticRegression)
ROC_curve_plot(y_test,y_pred_probs, 'LR')

In [ ]:
def ROC_curve_plots(y_test,y_pred_probs1, y_pred_probs2, y_pred_probs3, model1, model2, model3):
    fpr1, tpr1, _ = roc_curve(y_test, y_pred_probs1)
    auc1 = roc_auc_score(y_test, y_pred_probs1)
    fpr2, tpr2, _ = roc_curve(y_test, y_pred_probs2)
    auc2 = roc_auc_score(y_test, y_pred_probs2)
    fpr3, tpr3, _ = roc_curve(y_test, y_pred_probs3)
    auc3 = roc_auc_score(y_test, y_pred_probs3) 
    plt.figure(1,figsize=(12,8))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr1, tpr1, label=f'{model1} AUC={round(auc1,3)}')
    plt.plot(fpr2, tpr2, label=f'{model2} AUC={round(auc2,3)}')
    plt.plot(fpr3, tpr3, label=f'{model3} AUC={round(auc3,3)}')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()
    return 

In [ ]:
y_pred1, y_pred_probs1 = model_pred(X_train, y_train, X_test, y_test, LogisticRegression)
y_pred2, y_pred_probs2 = model_pred(X_train, y_train, X_test, y_test, RandomForestClassifier)
y_pred3, y_pred_probs3 = model_pred(X_train, y_train, X_test, y_test, GradientBoostingClassifier)

In [ ]:
ROC_curve_plots(y_test,y_pred_probs1, y_pred_probs2,y_pred_probs3, 'LR', 'RF','GB')

In [ ]:
cost_benefit = np.array([[0, 0], [-0.7, 0.3]]).T
def profit_curve(cost_benefit, predicted_probs, y_test):
    """Function to calculate list of profits based on supplied cost-benefit
    matrix and prediced probabilities of data points and thier true labels.
    Parameters
    ----------
    cost_benefit    : ndarray - 2D, with profit values corresponding to:
                                          -----------
                                          | TP | FP |
                                          -----------
                                          | FN | TN |
                                          -----------
    predicted_probs : ndarray - 1D, predicted probability for each datapoint
                                    in labels, in range [0, 1]
    labels          : ndarray - 1D, true label of datapoints, 0 or 1
    Returns
    -------
    profits    : ndarray - 1D
    thresholds : ndarray - 1D
    """
    n_obs = float(len(y_test))
    # Make sure that 1 is going to be one of our thresholds
    maybe_one = [] if 1 in predicted_probs else [1] 
    all_thresholds = maybe_one + sorted(predicted_probs, reverse=True)
    thresholds = all_thresholds[::50]
    profits = []
    for threshold in thresholds:
        y_predict = predicted_probs >= threshold
        confusion_matrix_ = confusion_matrix(y_test, y_predict, labels = [1,0])
        threshold_profit = np.sum(confusion_matrix_ * cost_benefit) / n_obs
        profits.append(threshold_profit)
    return np.array(profits), np.array(thresholds)

In [ ]:

def get_model_profits(model, cost_benefit, X_train, X_test, y_train, y_test):
    """Fits passed model on training data and calculates profit from cost-benefit
    matrix at each probability threshold.
    Parameters
    ----------
    model           : sklearn model - need to implement fit and predict
    cost_benefit    : ndarray - 2D, with profit values corresponding to:
                                          -----------
                                          | TP | FP |
                                          -----------
                                          | FN | TN |
                                          -----------
    X_train         : ndarray - 2D
    X_test          : ndarray - 2D
    y_train         : ndarray - 1D
    y_test          : ndarray - 1D
    Returns
    -------
    model_profits : model, profits, thresholds
    """
    model.fit(X_train, y_train)
    predicted_probs = model.predict_proba(X_test)[:, 1]
    profits, thresholds = profit_curve(cost_benefit, predicted_probs, y_test)

    return profits, thresholds

In [ ]:
profs, thrs = profit_curve(cost_benefit, y_pred_probs3, y_test)

In [ ]:
max_thr = thrs[np.argmax(profs)]
max_thr

In [ ]:
max_profs = np.max(profs)


In [ ]:
max_profs

In [ ]:
new_model = GradientBoostingClassifier()
new_model.fit(X_train, y_train)
#print(" done.")

#print('Convenience plot with ``partial_dependence_plots``')

features = [4,7,9,10]
fig, axs = plot_partial_dependence(new_model, X_train, features,figsize = (10, 10), 
                                   feature_names= [item for item in X.columns],
                                   n_jobs=3, grid_resolution=50)
fig.suptitle('Partial dependence plots')
plt.subplots_adjust(top=0.9, bottom = 0.2, wspace = 0.5)  # tight_layout causes overlap with suptitle

#print('Custom 3d plot via ``partial_dependence``')
#fig = plt.figure()
